In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_auc_score

%matplotlib inline

from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
from numba import jit

@jit
def mcc(tp, tn, fp, fn):
    sup = tp * tn - fp * fn
    inf = (tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)
    if inf==0:
        return 0
    else:
        return sup / np.sqrt(inf)

@jit
def eval_mcc(y_true, y_prob, show=False):
    idx = np.argsort(y_prob)
    y_true_sort = y_true[idx]
    n = y_true.shape[0]
    nump = 1.0 * np.sum(y_true) # number of positive
    numn = n - nump # number of negative
    tp = nump
    tn = 0.0
    fp = numn
    fn = 0.0
    best_mcc = 0.0
    best_id = -1
    prev_proba = -1
    best_proba = -1
    mccs = np.zeros(n)
    for i in range(n):
        # all items with idx < i are predicted negative while others are predicted positive
        # only evaluate mcc when probability changes
        proba = y_prob[idx[i]]
        if proba != prev_proba:
            prev_proba = proba
            new_mcc = mcc(tp, tn, fp, fn)
            if new_mcc >= best_mcc:
                best_mcc = new_mcc
                best_id = i
                best_proba = proba
        mccs[i] = new_mcc
        if y_true_sort[i] == 1:
            tp -= 1.0
            fn += 1.0
        else:
            fp -= 1.0
            tn += 1.0
    if show:
        y_pred = (y_prob >= best_proba).astype(int)
        score = matthews_corrcoef(y_true, y_pred)
        print(score, best_mcc)
        plt.plot(mccs)
        return best_proba, best_mcc, y_pred
    else:
        return best_mcc

def mcc_eval(y_prob, dtrain):
    y_true = dtrain.get_label()
    best_mcc = eval_mcc(y_true, y_prob)
    return 'MCC', best_mcc

In [5]:
#df1 = pd.read_hdf('feats_selected/train_init_761.hdf5','table')
df_train = pd.read_hdf('feats_selected/train_all_585_sel.hdf5','table')
#df_train = pd.concat([df1,df2],axis=1)

y = pd.read_csv('../../input_orig/train_numeric.csv', usecols=['Response'])

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=42, stratify=y.values)

In [6]:
del df_train
import gc
gc.collect()

dtrain = xgb.DMatrix(X_train, label=y_train)

del X_train
del y_train
gc.collect()

dval = xgb.DMatrix(X_test, label=y_test)

del X_test
del y_test
gc.collect()

14

In [7]:
dtrain.save_binary("all_feat_folder/train_all_585_sel.buffer")
dval.save_binary("all_feat_folder/val_all_585_sel.buffer")

In [2]:
dtrain = xgb.DMatrix('all_feat_folder/train_init_and_rest.buffer')
dval = xgb.DMatrix('all_feat_folder/val_init_and_rest.buffer')
y = pd.read_csv('../../input_orig/train_numeric.csv', usecols=['Response'])

In [ ]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.2,
    'max_depth': 24,
    'min_child_weight': 1,
    'gamma': 2,
    'base_score': prior,
    'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 22
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst = xgb.train(xgb_params, dtrain, num_boost_round=5000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1)

val_pred = bst.predict(dval, ntree_limit=bst.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst.get_score(fmap='', importance_type='gain')))

[0]	train-auc:0.933109	eval-auc:0.872744
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 20 rounds.
[1]	train-auc:0.865306	eval-auc:0.783704
[2]	train-auc:0.959411	eval-auc:0.874634
[3]	train-auc:0.964705	eval-auc:0.874836
[4]	train-auc:0.881739	eval-auc:0.786612


In [ ]:
def create_feature_map(features):
    outfile = open('xgb_all_no_selection.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

from operator import itemgetter
def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb_all_no_selection.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance

In [ ]:
#get_importance(bst1, bst1.feature_names)
create_feature_map(bst.feature_names)
bst.save_model('xgb_3237_all_039mcc_0906auc.model')

In [3]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'booster': 'dart',
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.05,
    'max_depth': 6,
    'min_child_weight': 1,
    'gamma': 1,
    'base_score': prior,
    'rate_drop': 0.2,
    'skip_drop': 0.5,
    #'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 22
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst1 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=20, 
                 evals=evals, verbose_eval=1)

val_pred = bst1.predict(dval, ntree_limit=bst1.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst1.get_score(fmap='', importance_type='gain')))

NameError: name 'y' is not defined

In [ ]:
prior = (np.sum(y) / (1.*len(y))).values[0]

xgb_params = {
    'booster': 'gbtree',
    'colsample_bytree': 1,
    'colsample_bylevel': 1,
    'subsample': 1,
    'learning_rate': 0.05,
    'max_depth': 12,
    'min_child_weight': 1,
    'gamma': 0,
    'base_score': prior,
    #'rate_drop': 0.2,
    #'skip_drop': 0.2,
    'tree_method': 'exact',
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'silent': 0,
    'seed': 12,
    'nthread': 22
}

evals  = [(dtrain,'train'), (dval,'eval')]
bst2 = xgb.train(xgb_params, dtrain, num_boost_round=50000, early_stopping_rounds=40, 
                 evals=evals, verbose_eval=1)

val_pred = bst2.predict(dval, ntree_limit=bst2.best_ntree_limit)
print(roc_auc_score(dval.get_label(), val_pred))
print(mcc_eval(val_pred, dval))
print(len(bst2.get_score(fmap='', importance_type='gain')))

[0]	train-auc:0.909679	eval-auc:0.890692
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 40 rounds.
[1]	train-auc:0.911273	eval-auc:0.891761
[2]	train-auc:0.917183	eval-auc:0.894994
[3]	train-auc:0.918496	eval-auc:0.894908
[4]	train-auc:0.919129	eval-auc:0.894846
[5]	train-auc:0.922331	eval-auc:0.895332
[6]	train-auc:0.923484	eval-auc:0.89631
[7]	train-auc:0.926535	eval-auc:0.89713
[8]	train-auc:0.927612	eval-auc:0.897592
[9]	train-auc:0.928422	eval-auc:0.898112
[10]	train-auc:0.931992	eval-auc:0.899009
[11]	train-auc:0.933225	eval-auc:0.898988
[12]	train-auc:0.933564	eval-auc:0.899269
[13]	train-auc:0.936351	eval-auc:0.898857
[14]	train-auc:0.937733	eval-auc:0.899495
[15]	train-auc:0.939197	eval-auc:0.89937
[16]	train-auc:0.940191	eval-auc:0.899381
[17]	train-auc:0.941099	eval-auc:0.90029
[18]	train-auc:0.942496	eval-auc:0.899988
[19]	train-auc:0.943014	eval-auc:0.900192
[20]	train-auc:0.944771	eval-auc: